# ANPR with OCR Implementation
License Plate Detection and Text Recognition using YOLOv8 and EasyOCR

In [ ]:
import subprocess
import sys

packages = ['ultralytics', 'opencv-python', 'pandas', 'numpy', 'matplotlib', 'seaborn', 'pillow', 'pyyaml', 'easyocr']

for pkg in packages:
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-q', pkg])

print('Dependencies installed')

In [ ]:
from ultralytics import YOLO
from pathlib import Path
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import json
import yaml
import easyocr
from datetime import datetime

np.random.seed(42)

print('Modules imported')

## Dataset Setup

In [ ]:
root = Path.cwd()
dataset_path = root / "anpr"
config_file = dataset_path / "data.yaml"

print(f'Project root: {root}')
print(f'Dataset: {dataset_path}')
print(f'Config: {config_file}')
print(f'Config exists: {config_file.exists()}')

In [ ]:
with open(config_file) as f:
    config = yaml.safe_load(f)

print('Configuration:')
print(json.dumps(config, indent=2))

In [ ]:
def count_images(path):
    if not path.exists():
        return 0
    return len(list(path.glob('*.jpg'))) + len(list(path.glob('*.png')))

train = count_images(dataset_path / "images" / "train")
val = count_images(dataset_path / "images" / "val")
test = count_images(dataset_path / "images" / "test")
total = train + val + test

print(f'Train: {train}')
print(f'Validation: {val}')
print(f'Test: {test}')
print(f'Total: {total}')

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(13, 5))

labels = ['Train', 'Validation', 'Test']
values = [train, val, test]
colors = ['#1f77b4', '#ff7f0e', '#2ca02c']

ax1.bar(labels, values, color=colors, edgecolor='black', linewidth=1.2)
ax1.set_ylabel('Images', fontsize=11)
ax1.set_title('Dataset Split', fontweight='bold')
ax1.grid(axis='y', alpha=0.3)

for i, v in enumerate(values):
    ax1.text(i, v + 300, str(v), ha='center', fontweight='bold')

ax2.pie(values, labels=labels, colors=colors, autopct='%1.1f%%', startangle=90)
ax2.set_title('Proportion', fontweight='bold')

plt.tight_layout()
plt.savefig('anpr_dataset_split.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
sample_path = dataset_path / "images" / "train"
samples = list(sample_path.glob('*.jpg'))[:6]

fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.ravel()

for idx, filepath in enumerate(samples):
    img = Image.open(filepath)
    axes[idx].imshow(img)
    axes[idx].set_title(filepath.name, fontsize=8)
    axes[idx].axis('off')

plt.tight_layout()
plt.savefig('anpr_samples.png', dpi=150, bbox_inches='tight')
plt.show()

## Training

In [ ]:
print('Loading model')
model = YOLO('yolov8m.pt')
print('Model loaded')

In [ ]:
print(f'Start time: {datetime.now()}')

results = model.train(
    data=str(config_file),
    epochs=100,
    imgsz=640,
    batch=16,
    patience=20,
    device=0,
    optimizer='SGD',
    momentum=0.937,
    weight_decay=0.0005,
    warmup_epochs=3.0,
    lr0=0.01,
    name='anpr_training',
    project='runs',
    exist_ok=True
)

print(f'End time: {datetime.now()}')

## Validation

In [ ]:
print('Running validation')
val_results = model.val()

print(f'mAP50: {val_results.box.map50:.4f}')
print(f'mAP50-95: {val_results.box.map:.4f}')

In [ ]:
print('Testing')
test_results = model.val(data=str(config_file), split='test')
print('Test complete')

In [ ]:
results_csv = Path('runs/anpr_training/results.csv')

if results_csv.exists():
    df = pd.read_csv(results_csv)
    print(f'Results shape: {df.shape}')
    print(df.head())

In [ ]:
if results_csv.exists():
    df = pd.read_csv(results_csv)
    
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    
    axes[0, 0].plot(df['epoch'], df['train/loss'], label='Train', linewidth=2)
    axes[0, 0].plot(df['epoch'], df['val/loss'], label='Val', linewidth=2)
    axes[0, 0].set_ylabel('Loss')
    axes[0, 0].set_title('Loss')
    axes[0, 0].legend()
    axes[0, 0].grid(alpha=0.3)
    
    axes[0, 1].plot(df['epoch'], df['metrics/mAP50'], label='mAP50', linewidth=2, marker='o', markersize=3)
    axes[0, 1].plot(df['epoch'], df['metrics/mAP50-95'], label='mAP50-95', linewidth=2, marker='s', markersize=3)
    axes[0, 1].set_ylabel('mAP')
    axes[0, 1].set_title('Mean Average Precision')
    axes[0, 1].legend()
    axes[0, 1].grid(alpha=0.3)
    
    axes[1, 0].plot(df['epoch'], df['metrics/precision'], linewidth=2)
    axes[1, 0].set_ylabel('Precision')
    axes[1, 0].set_title('Precision')
    axes[1, 0].grid(alpha=0.3)
    
    axes[1, 1].plot(df['epoch'], df['metrics/recall'], linewidth=2)
    axes[1, 1].set_ylabel('Recall')
    axes[1, 1].set_title('Recall')
    axes[1, 1].grid(alpha=0.3)
    
    plt.tight_layout()
    plt.savefig('anpr_metrics.png', dpi=150, bbox_inches='tight')
    plt.show()

## Inference Testing

In [ ]:
test_path = dataset_path / "images" / "test"
test_images = list(test_path.glob('*.jpg'))[:6]

print(f'Running inference on {len(test_images)} images')

pred_results = []
for img_file in test_images:
    res = model(str(img_file))
    pred_results.append(res[0])

print('Inference done')

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(17, 11))
axes = axes.ravel()

for idx, pred in enumerate(pred_results):
    annotated = pred.plot()
    img_rgb = annotated[..., ::-1]
    
    axes[idx].imshow(img_rgb)
    axes[idx].set_title(f'Detections: {len(pred.boxes)}')
    axes[idx].axis('off')

plt.tight_layout()
plt.savefig('anpr_predictions.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
print('Loading OCR reader')
ocr_reader = easyocr.Reader(['en'])
print('OCR reader loaded')

def extract_license_plate_text(image_path, detection_result):
    img = cv2.imread(str(image_path))
    extracted_plates = []
    
    if detection_result.boxes is not None and len(detection_result.boxes) > 0:
        for box in detection_result.boxes:
            x1, y1, x2, y2 = box.xyxy[0].cpu().numpy().astype(int)
            plate_region = img[y1:y2, x1:x2]
            
            if plate_region.size > 0:
                try:
                    ocr_result = ocr_reader.readtext(plate_region)
                    plate_text = ''.join([text[1] for text in ocr_result])
                    confidence = np.mean([text[2] for text in ocr_result]) if ocr_result else 0
                    
                    extracted_plates.append({
                        'text': plate_text,
                        'confidence': confidence,
                        'bbox': (x1, y1, x2, y2)
                    })
                except:
                    extracted_plates.append({
                        'text': 'Error',
                        'confidence': 0,
                        'bbox': (x1, y1, x2, y2)
                    })
    
    return extracted_plates

print('OCR function defined')

In [ ]:
print('Extracting license plate text with OCR')
ocr_results = {}

for img_file, pred in zip(test_images, pred_results):
    plates = extract_license_plate_text(img_file, pred)
    ocr_results[img_file.name] = plates
    print(f'{img_file.name}: {len(plates)} plates detected')

for filename, plates in ocr_results.items():
    print(f'\n{filename}:')
    for i, plate in enumerate(plates):
        print(f'  Plate {i+1}: {plate["text"]} (confidence: {plate["confidence"]:.2f})')

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
axes = axes.ravel()

for idx, (img_file, pred) in enumerate(zip(test_images, pred_results)):
    img = cv2.imread(str(img_file))
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    plates = ocr_results[img_file.name]
    
    for plate in plates:
        x1, y1, x2, y2 = plate['bbox']
        cv2.rectangle(img_rgb, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(img_rgb, plate['text'], (x1, y1 - 10),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
    
    axes[idx].imshow(img_rgb)
    title = f'{img_file.name}\nPlates: {len(plates)}'
    if plates:
        title += f'\nText: {plates[0]["text"]}'
    axes[idx].set_title(title, fontsize=9, fontweight='bold')
    axes[idx].axis('off')

plt.tight_layout()
plt.savefig('anpr_with_ocr.png', dpi=150, bbox_inches='tight')
plt.show()

print('OCR visualization complete')

## Model Export

In [ ]:
save_path = 'yolo_ANPR.pt'
model.save(save_path)

size_mb = Path(save_path).stat().st_size / 1024 / 1024

print(f'Model saved: {save_path}')
print(f'Size: {size_mb:.2f} MB')

In [ ]:
print('\n' + '-'*50)
print('ANPR TRAINING SUMMARY')
print('-'*50)
print(f'Model: YOLOv8 Medium')
print(f'Task: License Plate Detection')
print(f'Resolution: 640x640')
print(f'Classes: 1')
print(f'Epochs: 100')
print(f'Batch: 16')
print(f'\nDataset:')
print(f'Total: {total}')
print(f'Train: {train}')
print(f'Val: {val}')
print(f'Test: {test}')
print(f'\nModel File: {save_path}')
print(f'File Size: {size_mb:.2f} MB')
print('-'*50)